In [ ]:
%load_ext autoreload
%autoreload 2

# routerchain

> This module contains the development of a faster router chain using openai functions

In [ ]:
#| default_exp routerchain

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from dotenv import load_dotenv, find_dotenv


In [ ]:
#| hide
load_dotenv('../.env')

True

In [ ]:
#| exports

from typing import Any, Dict, List, Mapping, NamedTuple, Optional
from pydantic import Extra, BaseModel, Field

from langchain.chains.base import Chain
from langchain.chains import LLMChain, ConversationChain, RetrievalQA

from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ( PromptTemplate, 
                               ChatPromptTemplate, 
                               SystemMessagePromptTemplate,
                               HumanMessagePromptTemplate)

from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import BaseLLM

from fastcore.all import *


# MutliRouteChain usinf OpenAI Functions

This class is the one that is routing the different chains, it receives the input, call the router chain.
The Router chain will return the name of the next chain to call, if he doesn't find a next chain, it returns None.

In [ ]:
#| exports
class MultiRouteChain_openAIfunc(Chain):
    """Use a single chain to route an input to one of multiple candidate chains."""

    router_chain: LLMChain
    """Chain that routes inputs to destination chains."""
    destination_chains: Mapping[str, Chain]
    """Chains that return final answer to inputs."""
    default_chain: Chain
    """Default chain to use when none of the destination chains are suitable."""
    silent_errors: bool = False
    """If True, use default_chain when an invalid destination name is provided. 
    Defaults to False."""

    class Config:
        """Configuration for this pydantic object."""
        Extra.allow 
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the router chain prompt expects.

        :meta private:
        """
        return ['input']

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return []

    def _call(
        self,
        inputs: Dict[str, Any],
    ) -> Dict[str, Any]:
        try:
            route =  self.router_chain(inputs)
            route = route['function']
            print(f"Next Function to call {route}")

        except:
            route = None
            
        if not route:
            return self.default_chain(
                {
                    'input': inputs['input']
                }
            )
        elif route.name in self.destination_chains:
            return self.destination_chains[route.name](
                route.next_inputs
            )
        elif self.silent_errors:
            return self.default_chain(route.next_inputs, callbacks=callbacks)
        else:
            raise ValueError(
                f"Received invalid destination chain name '{route.destination}'"
            )

    async def _acall(
        self,
        inputs: Dict[str, Any],
    ) -> Dict[str, Any]:
        try:
            route = self.router_chain.run(inputs)
            route = route['function']
            print(f"Next Function to call {route}")
        except :
            route = None

        if not route:
            return await self.default_chain.acall(
                {
                    'input': inputs['input']
                }
            )
        elif route.name in self.destination_chains:
            return await self.destination_chains[route.name].acall(
                {
                    'input':route.next_inputs
                }
            )
        elif self.silent_errors:
            return await self.default_chain.acall(
                {
                    'input': route.next_inputs
                }
            )
        else:
            raise ValueError(
                f"Received invalid destination chain name '{route.destination}'"
            )

Here is the Router chain declaration. It recieves the input and decide, using openAI functions, which would be the next chain to call

## Classes for chains

In [ ]:
#| exports
class Conversational_chain(BaseModel):
    """Tool to continue a friendly conversation"""
    name:str = Field(default='conversational chain',
                    description="The name of the tool to help the user.",
                    enum=['conversational chain'])
    next_inputs:str = Field(...,description="the same message recieved.")
    

In [ ]:
#| exports
class Retrieval_chain(BaseModel):
    """Tool to continue a friendly conversation"""
    name:str = Field(default='retrieval chain',
                    description="The name of the tool to help the user.",
                    enum=['retrieval chain'])
    next_inputs:str = Field(...,description="A detailed question to search in a vectore database the answer to the question recieved")
    

## Creation of the Router Chain

In [ ]:
#| exports
def create_router_template():
    return """"
Given a raw text input to a large language model and the chat history, select the best tool to use with the input. You will be given the names of the available tools and a description of what they can do.



<< CHAT HISTORY >>
{{history}}
<< END >>

Tip: Make sure to answer in the correct format
"""

In [ ]:
#| exports
def create_router_chain( 
                    llm:BaseLLM, #LLM model instantiated
                    verbose:bool = False #Boolean to activate the verbose
                       )->(LLMChain, dict): #Returns an LLMChain and a dictionary with the destination chains
    
    router_template=create_router_template()
    
    #Here are the chain options, The name would be the name of the function/chain and the chain is the function to be called.
    model_infos = [
    # {
    #     "name": "retrieval chain",
    #     'chain': RetrievalQA.from_llm(llm=llm) ,
    # },
    {
        "name": "conversational chain",
        'chain': ConversationChain(
                    llm=llm,
                    output_key="response" ,
                    verbose=verbose
        ),

    },
    ]

    destination_chains = {}
    for p_info in model_infos:
        name = p_info["name"]
        chain = p_info['chain']
        destination_chains[name] = chain
        
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(
                template="""
                You are a world class algorithm for finding the next tool to be used to answers the human question.
                """),
                HumanMessagePromptTemplate.from_template(
                    template="""
                    Use the given format to extract information from the following input: {input}
                    
                    Tip: Make sure to answer in the correct format""")
        ]
    )
    #Here is the creation of the chain with openai functions. The first argument is a list of pydantic classes that were declared above and contains the information of the chain, name and what they do
    chain = create_openai_fn_chain([Conversational_chain], llm, prompt, verbose=verbose)
    return chain, destination_chains

## Now we create the Multirouter chain

In [ ]:
#| exports
def create_MultiChianRouter_o(
                    llm:BaseLLM, #LLM instance
                    verbose:bool = False #Boolean to activate the verbose
    )->MultiRouteChain_openAIfunc: #It returns a multirouter chain
    
    router_chain, destination_chains = create_router_chain(
                    llm, 
                    verbose=verbose
    )
    # Here we created a memory that will be shared by all the chains that use memory, or we can give individual memories
    memory = ConversationBufferWindowMemory(
                                                    memory_key='history', 
                                                    output_key='response'
                                                    )
    rChain= MultiRouteChain_openAIfunc(
                router_chain=router_chain,
                default_chain=ConversationChain(
                    llm=llm,
                    output_key="response" ,
                    verbose=verbose,
                    memory=memory,
                ),
                    destination_chains=destination_chains, 
                    verbose = verbose,
                    memory=memory
            )
    
    return rChain

In [ ]:
#| eval: false
mChain= create_MultiChianRouter_o(
    llm=ChatOpenAI(),
     verbose=True
)

In [ ]:
#| eval: false
mChain('Hello')



> Entering new MultiRouteChain_openAIfunc chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
                You are a world class algorithm for finding the next tool to be used to answers the human question.
                
Human: 
                    Use the given format to extract information from the following input: Hello
                    
                    Tip: Make sure to answer in the correct format

> Finished chain.
Next Function to call name='conversational chain' next_inputs='Hello'


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello
AI: Hi there! How can I assist you today?
Human: Hello
AI:

> Finished chain.

> Finished chain.


{'input': 'Hello',
 'history': 'Human: Hello\nAI: Hi there! How can I assist you today?',
 'response': 'Hello! How can I assist you today?'}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
